## **Installing Dependencies**

In [ ]:
!pip install datasets bitsandbytes torch transformers accelerate llama-index llama-index-embeddings-huggingface peft auto-gptq optimum selfcheckgpt

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 28.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.7 MB/s eta 0:00:00
  

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `thesis` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `thesis`


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings("ignore")

## **Importing Libraries**

In [ ]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import numpy as np
import torch
import ast
import spacy
import random
from collections import Counter
import re
import pandas as pd
from tqdm import tqdm
from typing import Dict, List, Set, Tuple, Union

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, VectorStoreIndex, Document
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    classification_report
)

from selfcheckgpt.modeling_selfcheck import SelfCheckLLMPrompt, SelfCheckMQAG, SelfCheckBERTScore, SelfCheckNgram, SelfCheckNLI

In [ ]:
torch.manual_seed(28)

In [ ]:
# Sets spacy for processing the text
nlp = spacy.load("en_core_web_sm")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## **Load the data**

In [ ]:
hallu_felm_sample = pd.read_csv('/content/drive/MyDrive/Ceccarelli_MasterThesis&Internship/Master Thesis/felm_w_knowledge.csv')
hallu_felm_examples = pd.read_csv('/content/drive/MyDrive/Ceccarelli_MasterThesis&Internship/Master Thesis/felm_w_knowledge_examples.csv')

In [ ]:
# Check the class distribution
Counter(hallu_felm_sample['hallucination'])

Counter({True: 50, False: 50})

In [ ]:
hallu_felm_sample = Dataset.from_pandas(hallu_felm_sample) # Convert to Pandas dataframe
hallu_felm_sample[0]

{'prompt': 'This Jewish American studied at NYU, was born and lived in New York City, and became famous in the mid 1980s for a controversial act. His name is "Bernie" what?',
 'response': 'The person you are referring to is Bernie Goetz, also known as the "Subway Vigilante".',
 'hallucination': True,
 '__index_level_0__': 233,
 'knowledge_url': 'https://arxiv.org/html/2306.03341v6',
 'knowledge': '\n\n\n\nInference-Time Intervention: Eliciting Truthful Answers from a Language Model\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n1 Introduction\n2 Related Work\n\n3 Inference-Time Intervention for Eliciting Truthful Answers\n\n3.1 Setup\n3.2 Probing for “Truthfulness”\n3.3 Inference-Time Intervention\n\n\n\n4 Experiments\n\n4.1 Evaluation on TruthfulQA\n4.2 Experimental Baseline Comparisons\n4.3 Experimental Results\n\n\n\n5 Analysis\n\n5.1 Results Across TruthfulQA Categories\n5.2 Computational Efficiency\n5.3 Generalization of ITI beyond TruthfulQA\n5.4 Varying Training Set Size and Intervention Streng

In [ ]:
hallu_felm_examples = Dataset.from_pandas(hallu_felm_examples) # Convert to Pandas dataframe
hallu_felm_examples[0]

{'prompt': 'Have humans flown close to the sun?',
 'response': "No, humans have never flown close to the sun. The sun's surface temperature is about 9,900 degrees Fahrenheit (5,500 degrees Celsius), which is hot enough to melt lead. Additionally, the sun's extreme gravitational pull would make it nearly impossible for a human spacecraft to approach it without being pulled in and destroyed. The closest humans have been to the sun is through unmanned space probes such as NASA's Parker Solar Probe, which was launched in 2018 and has flown as close as 3.8 million miles (6.1 million kilometers) to the sun's surface, but even that distance is still far enough to ensure the spacecraft and its instruments are not destroyed by the sun's intense heat and radiation.",
 'hallucination': False,
 'knowledge_url': 'https://www.reddit.com/r/space/comments/3lft2d/what_is_the_closest_a_human_being_has_ever_been/',
 'knowledge': "You've been blocked by network security.To continue, log in to your Reddit 

In [ ]:
hallu_felm_sample

Dataset({
    features: ['prompt', 'response', 'hallucination', '__index_level_0__', 'knowledge_url', 'knowledge'],
    num_rows: 100
})

In [ ]:
# Basic cleaning of the knowledge retrieved
def clean_text(example):
  text = example['knowledge']
  try:
    text = text.replace('\n','')
    text = text.replace('\t', '')
    text = text.replace('\r', '')
    text = text.replace('\n*', '')
    example['knowledge'] = text
  except:
    pass
  return example

hallu_felm_sample = hallu_felm_sample.map(lambda x: clean_text(x))
hallu_felm_examples = hallu_felm_examples.map(lambda x: clean_text(x))

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

## **Set embedding model and settings for RAG**

In [ ]:
# Retrieves the embedding model for indexing and retrieving the knowledge for RAG
Settings.embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-base-en-v1.5", trust_remote_code = True)

Settings.llm = None
Settings.chunk_size = 128 # Number of characters within each chunk
Settings.chunk_overlap = 25 # Number of characters that overlaps for not truncating the chunk

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

LLM is explicitly disabled. Using MockLLM.


## **Load the model**

In [ ]:
# Configures bitsandbytes for a 4-bit quantization of the LLM with double quantization, for the efficiency
# load_in_4bit: Enables the 4-bit quantization of the model
# bnb_4bit_use_double_quant: Enables double quantization that further decreases the computational efforts
# bnb_4bit_quant_type: Type of quantization used
# bnb_4bit_compute_dtype: Define the type of data used during computation in the inference phase

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
# Retrieve the model and the corresponding tokenizer from Hugging Face using the method AutoModelForCausalLM to use the generative part of the LLM

model_id = "meta-llama/Llama-3.2-3B-Instruct"

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map='auto')

tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
class SelfCheckLLMPrompt_Quantized:
    """
    SelfCheckGPT (LLM Prompt): Checking LLM's text against its own sampled texts via open-source LLM prompting
    """
    def __init__(
        self,
        model: str = None,
        device = None,
        quantization = True
    ):
        model = model if model is not None else "meta-llama/Llama-3.2-3B-Instruct"
        """
        Gives the chanche to load a quantized model from Hugging Face.
        """
        if quantization is not True:
            self.tokenizer = AutoTokenizer.from_pretrained(model)
            self.model = AutoModelForCausalLM.from_pretrained(model, torch_dtype="auto")
            self.model.eval()
        else:
          bnb_config = BitsAndBytesConfig(
          load_in_4bit=True,
          bnb_4bit_use_double_quant=True,
          bnb_4bit_quant_type="nf4",
          bnb_4bit_compute_dtype=torch.bfloat16
          )
          self.tokenizer = AutoTokenizer.from_pretrained(model)
          self.model = AutoModelForCausalLM.from_pretrained(model, quantization_config=bnb_config, torch_dtype="auto")
          self.model.eval()
        if device is None:
            device = torch.device("cpu")
        self.model.to(device)
        self.device = device
        self.prompt_template = "Context: {context}\n\nSentence: {sentence}\n\nIs the sentence supported by the context above? Answer Yes or No.\n\nAnswer: "
        self.text_mapping = {'yes': 0.0, 'no': 1.0, 'n/a': 0.5}
        self.not_defined_text = set()
        print(f"SelfCheck-LLMPrompt ({model}) initialized to device {device}")

    def set_prompt_template(self, prompt_template: str):
        self.prompt_template = prompt_template

    @torch.no_grad()
    def predict(
        self,
        sentences: List[str],
        sampled_passages: List[str],
        verbose: bool = False,
    ):
        """
        This function takes sentences (to be evaluated) with sampled passages (evidence), and return sent-level scores
        :param sentences: list[str] -- sentences to be evaluated, e.g. GPT text response spilt by spacy
        :param sampled_passages: list[str] -- stochastically generated responses (without sentence splitting)
        :param verson: bool -- if True tqdm progress bar will be shown
        :return sent_scores: sentence-level scores
        """
        num_sentences = len(sentences)
        num_samples = len(sampled_passages)
        scores = np.zeros((num_sentences, num_samples))
        disable = not verbose
        for sent_i in tqdm(range(num_sentences), disable=disable):
            sentence = sentences[sent_i]
            for sample_i, sample in enumerate(sampled_passages):

                # this seems to improve performance when using the simple prompt template
                sample = sample.replace("\n", " ")

                prompt = self.prompt_template.format(context=sample, sentence=sentence)
                inputs = self.tokenizer(prompt, return_tensors="pt").to(self.device)
                generate_ids = self.model.generate(
                    inputs.input_ids,
                    max_new_tokens=5,
                    do_sample=False, # hf's default for Llama2 is True
                )
                output_text = self.tokenizer.batch_decode(
                    generate_ids, skip_special_tokens=True,
                    clean_up_tokenization_spaces=False
                )[0]
                generate_text = output_text.replace(prompt, "")
                score_ = self.text_postprocessing(generate_text)
                scores[sent_i, sample_i] = score_
        scores_per_sentence = scores.mean(axis=-1)
        return scores_per_sentence

    def text_postprocessing(
        self,
        text,
    ):
        """
        To map from generated text to score
        Yes -> 0.0
        No  -> 1.0
        everything else -> 0.5
        """
        # tested on Llama-2-chat (7B, 13B) --- this code has 100% coverage on wikibio gpt3 generated data
        # however it may not work with other datasets, or LLMs
        text = text.lower().strip()
        if text[:3] == 'yes':
            text = 'yes'
        elif text[:2] == 'no':
            text = 'no'
        else:
            if text not in self.not_defined_text:
                print(f"warning: {text} not defined")
                self.not_defined_text.add(text)
            text = 'n/a'
        return self.text_mapping[text]

In [ ]:
# Text generation pipeline

# max_new_tokens: Maximum number of new tokens generated
# temperature: Controls the randomness of the new tokens

pipe = pipeline("text-generation",
                model=model,
                tokenizer=tokenizer,
                device_map="auto",
                temperature=1.0,
                do_sample=True,
                max_new_tokens=128,
                return_full_text=False,
)

In [ ]:
# Initializes the variants of SelfCheckGPT to be used

selfcheck_llm = SelfCheckLLMPrompt_Quantized(device=device, model = model_id)
selfcheck_nli = SelfCheckNLI(device=device)
selfcheck_bertscore = SelfCheckBERTScore()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

SelfCheck-LLMPrompt (meta-llama/Llama-3.2-3B-Instruct) initialized to device cuda


tokenizer_config.json:   0%|          | 0.00/400 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/883 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

SelfCheck-NLI initialized to device cuda
SelfCheck-BERTScore initialized


## **SelfCheckGPT**

In [ ]:
# Applies SelfCheckGPT to the instances

def get_selfcheckgpt_scores(example):
  N = 5 # Number of sampled responses
  if example['knowledge']:

    top_k = 7 # Number of top relevant chunks to be retrieved
    documents = [Document(text = example['knowledge'])] # Transform the knowledge into a Document
    index = VectorStoreIndex.from_documents(documents) # Indexes the knowledge
    retriever = VectorIndexRetriever(index=index, # Define the retriever
                                   similarity_top_k = top_k)
    query_engine = RetrieverQueryEngine(retriever=retriever, # Retrieves the top k relevant document to the query
                                      node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)])
    response = query_engine.query(example['prompt']) # Define the query as the prompt that generated the response
    knowledge = '\n\n'

# Adds the knowledge to the prompt given to the LLM

    for k in range(min(top_k, len(response.source_nodes))):
      knowledge = knowledge + response.source_nodes[k].text + '\n\n'
  else:
    knowledge = ''

# Prompt for response sampling

  prompt = f"""Based on your knowledge and on the context provided, answer the following question giving as much detail as you can.
  Question: {example['prompt']}
  Context: {knowledge}
  Answer:"""

  samples = pipe([prompt] * N, do_sample=False, max_new_tokens=128, return_full_text=False) # Gives the prompt to the LLM that returns N responses to the prompt
  samples = [sample[0]["generated_text"] for sample in samples]
  sentences = [
    sent.text.strip() for sent in nlp(example['response']).sents # Split the original response in sentences
              ]
  text_scores_llm = np.mean(selfcheck_llm.predict( # Computes the mean of the LLM prompt scores obtained in each sentence, to compute the total score for the response
                            sentences = sentences,
                            sampled_passages = samples))
  text_scores_nli = selfcheck_nli.predict( # Computes the mean of the NLI scores obtained in each sentence, to compute the total score for the response
    sentences = sentences,
    sampled_passages = samples)
  text_scores_bertscore = selfcheck_bertscore.predict( # Computes the mean of the BERTScores obtained in each sentence, to compute the total score for the response
    sentences = sentences,
    sampled_passages = samples)
  print([np.mean(text_scores_llm), np.mean(text_scores_nli), np.mean(text_scores_bertscore)])

  return {                                            # Saves in the dataset the scores obtained
      'scores_llm': np.mean(text_scores_llm),
      'scores_nli': np.mean(text_scores_nli),
      'scores_bertscore': np.mean(text_scores_bertscore)
  }

In [ ]:
# Apply SelfCheckGPT with knowledge

hallu_felm_sample = hallu_felm_sample.map(get_selfcheckgpt_scores)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

[1.0, 0.9891881346702576, 0.9261990413069725]
[0.3333333333333333, 0.3098713726018711, 0.604726642370224]
[0.5, 0.023387513763736933, 0.49748389422893524]
[0.5, 0.43072631416725926, 0.4166209250688553]
[0.0, 0.36009053927409695, 0.46124356612563133]
[0.0, 0.7553803697228432, 0.7238877326250076]
[0.5, 0.2865887749164055, 0.5906084083641568]
[0.0, 0.16917366086272523, 0.4933147244155407]
[0.0, 0.02133917125562827, 0.520449365178744]
[0.0, 0.10236761435808148, 0.572593292593956]
[0.6, 0.021071144193410874, 0.7749319314956665]
[0.5, 0.5113551402464509, 0.5767640024423599]
[0.0, 0.4282001913525164, 0.3217555433511734]
[0.0, 0.0487526876386255, 0.5523958280682564]
[0.0, 0.46337179839611053, 0.7136559337377548]
[0.0, 0.05481648177956231, 0.6549397140741349]
[0.0, 0.0007266232278198004, 0.06462723016738892]
[0.3333333333333333, 0.7653273320393055, 0.7723319306969643]
[0.5, 0.7735078036785126, 0.5300810933113098]
[0.0, 0.32532691955566406, 0.6452300027012825]
[0.0, 0.2151999790148693, 0.5902973

In [ ]:
hallu_felm_sample

Dataset({
    features: ['prompt', 'response', 'hallucination', '__index_level_0__', 'knowledge_url', 'knowledge', 'scores_llm', 'scores_nli', 'scores_bertscore'],
    num_rows: 100
})

In [ ]:
def find_best_threshold(dataset, continuous_col, binary_col, num_thresholds=200):

    """
    Find the optimal threshold to map the scores in 0/1, maximizing the accuracy.

    :param dataset: DataFrame containing the scores and the binary labels
    :param continuous_col: Column name with scores
    :param binary_col: Column name with binary labels (ground truths)
    :param num_thresholds: Number of thresholds to test
    :return: Best threshold and corresponding accuracy
    """
    y_true = np.array(dataset[binary_col])  # Ground truths
    y_scores = np.array(dataset[continuous_col])  # SelfCheckGPT scores

    # Generate thresholds between the minimum and maximum values of the scores
    thresholds = np.linspace(y_scores.min(), y_scores.max(), num_thresholds)

    best_threshold = None
    best_accuracy = 0

    # Tests each tresholds, maximizing the accuracy
    for threshold in thresholds:
        y_pred = (y_scores <= threshold).astype(int)  # Maps in 0/1
        acc = accuracy_score(y_true, y_pred)  # Compute accuracy

        if acc > best_accuracy:
            best_accuracy = acc
            best_threshold = threshold

    return best_threshold, best_accuracy

In [ ]:
# Computes best thresholds and corresponding accuracy

best_threshold_llm, best_accuracy_llm = find_best_threshold(hallu_felm_sample, 'scores_llm', 'hallucination')
best_threshold_nli, best_accuracy_nli = find_best_threshold(hallu_felm_sample, 'scores_nli', 'hallucination')
best_threshold_bertscore, best_accuracy_bertscore = find_best_threshold(hallu_felm_sample, 'scores_bertscore', 'hallucination')

## **Saving dataframes with metrics**

Convert the results in a dataframe and save it on Google Drive.

In [ ]:
metrics = {
    'classification type' : ['llm_prompt', 'nli', 'bertscore'],
    'best_threshold' : [best_threshold_llm, best_threshold_nli, best_threshold_bertscore],
    'best_accuracy' : [best_accuracy_llm, best_accuracy_nli, best_accuracy_bertscore]
}

metrics_df = pd.DataFrame(metrics)
metrics_df

,classification type,best_threshold,best_accuracy
0,llm_prompt,0.376884,0.69
1,nli,0.306543,0.71
2,bertscore,0.662099,0.58


In [ ]:
path = '/content/drive/MyDrive/Ceccarelli_MasterThesis&Internship/Master Thesis/Results'

with open(path + "/metrics_selfcheckgpt_w_knowledge_felm.csv", "w") as f:
    metrics_df.to_csv(f, index=False)